In [1]:
# 必要なライブラリのインポート
import tensorflow as tf
import numpy as np


# 変数の定義
dim = 5
with tf.name_scope('X'):
    x = tf.placeholder(tf.float32, [None, dim + 1])
with tf.name_scope('W'):
    w = tf.Variable(tf.zeros([dim+1,1]))
with tf.name_scope('multi'):
    y = tf.matmul(x,w)
with tf.name_scope('T'):
    t = tf.placeholder(tf.float32, [None, 1])
sess = tf.Session()

# 損失関数と学習メソッドの定義
with tf.name_scope('loss'):
    loss = tf.reduce_sum(tf.square(y - t))
with tf.name_scope('train'):
    train_step = tf.train.AdamOptimizer().minimize(loss)
    
# TensorBoardで追跡する変数を定義
with tf.name_scope('summary'):
    tf.summary.scalar('loss', loss)
    merged = tf.summary.merge_all()
    writer = tf.summary.FileWriter('./logs', sess.graph)
    
# セッションの初期化と入力データの準備
sess.run(tf.global_variables_initializer())

train_t = np.array([5.2, 5.7, 8.6, 14.9, 18.2, 20.4,25.5, 26.4, 22.8, 17.5, 11.1, 6.6])
train_t = train_t.reshape([12,1])
train_x = np.zeros([12, dim+1])
for row, month in enumerate(range(1, 13)):
    for col, n in enumerate(range(0, dim+1)):
        train_x[row][col] = month**n

# 学習

i = 0
for _ in range(100000):
    i += 1
    sess.run(train_step, feed_dict={x: train_x, t: train_t})
    if i % 10000 == 0:
        loss_val = sess.run(loss, feed_dict={x: train_x, t: train_t})
        print('Step: %d, Loss: %f' % (i, loss_val))

Step: 10000, Loss: 236.498520
Step: 20000, Loss: 15.957458
Step: 30000, Loss: 15.397224
Step: 40000, Loss: 14.942535
Step: 50000, Loss: 14.537667
Step: 60000, Loss: 14.162298
Step: 70000, Loss: 15.303836
Step: 80000, Loss: 13.546728
Step: 90000, Loss: 575.119751
Step: 100000, Loss: 12.884917


In [ ]:
# tensorboard --logdir=./logs

In [ ]:
# http://vigilant:6006/#scalars

In [4]:
import os
from urllib.request import urlretrieve
from urllib.parse import urlparse
from zipfile import ZipFile

def download_file(url, output_dir, overwrite=False):
  # URLからファイル名を取得
  parse_result = urlparse(url)
  file_name = os.path.basename(parse_result.path)
  # 出力先ファイルパス
  destination = os.path.join(output_dir, file_name)

  # 無意味なダウンロードを防ぐため、上書き（overwrite）の指定か未ダウンロードの場合のみダウンロードを実施する
  if overwrite or not os.path.exists(destination):
    # 出力先ディレクトリの作成
    os.makedirs(output_dir)
    # ダウンロード
    urlretrieve(url, destination)

  return destination

zip_file = download_file('https://archive.ics.uci.edu/ml/machine-learning-databases/00360/AirQualityUCI.zip', 'UCI_data/')

In [5]:
import pandas as pd

with ZipFile(zip_file) as z:
  with z.open('AirQualityUCI.xlsx') as f:
    air_quality = pd.read_excel(
      f,
      index_col=0, parse_dates={'DateTime': [0, 1]}, #1
      na_values=[-200.0],                            #2
      convert_float=False                            #3
    )